In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 채널 차원을 추가합니다.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [4]:
train_ds_ = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

train_ds = train_ds_.take(int(120))


test_ds_ = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)
test_ds = test_ds_.take(int(40))

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='swish')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='swish')
        #self.d1 = Dense(50, activation='swish')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()


loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    
EPOCHS = 300

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

(32, 28, 28, 1)


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node my_model/conv2d/Conv2D (defined at <ipython-input-4-121f083c9efe>:20) ]] [Op:__inference_train_step_587]

Errors may have originated from an input operation.
Input Source operations connected to node my_model/conv2d/Conv2D:
 Cast (defined at <ipython-input-4-121f083c9efe>:41)

Function call stack:
train_step


In [237]:
# take data for its key
buffer = {}
#print(len(y_train))
for idx, y_ in enumerate(y_train):
    if y_ in buffer.keys():
        buffer[y_].append(x_train[idx])
    else:
        buffer[y_] = [x_train[idx]]
        
        
num_same = 200
for key, value in buffer.items():
    value_ = np.array(value)
    choice = np.random.choice(value_.shape[0], num_same, replace=False)
    choosen = value_[choice]
    buffer[key] = choosen

# take same key
inputs = np.empty([1, 2, 28, 28, 1])
outputs = np.empty([1])
ratio_same = 0.5
for key, value in buffer.items():
    for key_, value_ in buffer.items():
        # take same cases
        if key == key_:
            choice1 = np.random.choice(value.shape[0], int(num_same * ratio_same), replace=False)
            choosen1 = np.expand_dims(value[choice1], axis=1)
            choice2 = np.random.choice(value_.shape[0], int(num_same * ratio_same), replace=False)
            choosen2 = np.expand_dims(value_[choice2], axis=1)   
            
            tmp_same = np.concatenate((choosen1, choosen2), axis=1)
            
            
            inputs = np.concatenate((inputs, tmp_same), axis=0)
            
            outputs = np.concatenate(   (outputs,   np.full(    (int(num_same * ratio_same)), 1     )  ), axis=0          )
            
            
        # take different cases
        
        else:
            choice1 = np.random.choice(value.shape[0], int(num_same * ratio_same / (len(buffer.keys()) - 1)), replace=False)
            choosen1 = np.expand_dims(value[choice1], axis=1)
            choice2 = np.random.choice(value_.shape[0], int(num_same * ratio_same / (len(buffer.keys()) - 1)), replace=False)
            choosen2 = np.expand_dims(value_[choice2], axis=1)  
            
            tmp_dif = np.concatenate((choosen1, choosen2), axis=1)
            
            inputs = np.concatenate((inputs, tmp_dif), axis=0)
            
            outputs = np.concatenate(   (outputs,   np.full(    (int(num_same * ratio_same / (len(buffer.keys()) - 1))), 0     )  ), axis=0          )

#print(inputs.shape)
#print(outputs.shape)


inputs = np.delete(inputs, 0, axis=0)
outputs = np.delete(outputs, 0, axis=0)

#print(inputs.shape)
#print(outputs.shape)


dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs)).shuffle(inputs.shape[0], reshuffle_each_iteration=True)

dataset_batched = dataset.batch(32, drop_remainder=True)
numberOfData = dataset_batched.cardinality().numpy()
trainDataset = dataset_batched.take(int(numberOfData * 0.7))
validDataset = dataset_batched.skip(int(numberOfData * 0.7))


In [238]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='swish')
        self.flatten = Flatten()
        self.d1 = Dense(50, activation='swish')
        self.d2 = Dense(1, activation='sigmoid')

    def call(self, x):
        input_1 = x[:,0,:,:]
        input_2 = x[:,1,:,:]
        #print(input_1.shape)
        x = self.conv1(input_1)
        x_1 = self.flatten(x)
        
        x = self.conv1(input_2)
        x_2 = self.flatten(x)
        #print(x_2.shape)
        x = tf.concat([x_1, x_2], 1)
        #print(x.shape)
        
        x = self.d1(x)
        return self.d2(x)
    
    
model = MyModel()

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean()
train_acc = tf.keras.metrics.BinaryAccuracy()

valid_loss = tf.keras.metrics.Mean()
valid_acc = tf.keras.metrics.BinaryAccuracy()

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        #print(predictions.shape)
        #print(labels.shape)
        loss = loss_object(labels, predictions)
        train_loss(loss)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_acc.update_state(labels, predictions)
    

@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    valid_loss(t_loss)
    valid_acc.update_state(labels, predictions)
    

EPOCHS = 300

for epoch in range(EPOCHS):
    for images, labels in trainDataset:
        train_step(images, tf.expand_dims(labels, axis=-1))
        
    for images, labels in validDataset:
        test_step(images, tf.expand_dims(labels, axis=-1))
        
    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_acc.result()*100,
                         valid_loss.result(),
                         valid_acc.result()*100))

에포크: 1, 손실: 0.7121461629867554, 정확도: 54.433135986328125, 테스트 손실: 0.6743165850639343, 테스트 정확도: 59.04605484008789
에포크: 2, 손실: 0.6893221139907837, 정확도: 57.594478607177734, 테스트 손실: 0.6488251686096191, 테스트 정확도: 62.417762756347656
에포크: 3, 손실: 0.6579790115356445, 정확도: 62.06395721435547, 테스트 손실: 0.6087519526481628, 테스트 정확도: 67.26973724365234
에포크: 4, 손실: 0.6216211318969727, 정확도: 65.77034759521484, 테스트 손실: 0.5657430291175842, 테스트 정확도: 70.84703826904297
에포크: 5, 손실: 0.5859260559082031, 정확도: 68.98255920410156, 테스트 손실: 0.5298242568969727, 테스트 정확도: 73.42105865478516
에포크: 6, 손실: 0.5503694415092468, 정확도: 71.71753692626953, 테스트 손실: 0.4962003529071808, 테스트 정확도: 76.17872619628906
에포크: 7, 손실: 0.5185781121253967, 정확도: 74.07599639892578, 테스트 손실: 0.4647456705570221, 테스트 정확도: 78.5244369506836
에포크: 8, 손실: 0.48705628514289856, 정확도: 76.33539581298828, 테스트 손실: 0.43681883811950684, 테스트 정확도: 80.38651275634766
에포크: 9, 손실: 0.45835766196250916, 정확도: 78.24612426757812, 테스트 손실: 0.41018903255462646, 테스트 정확도: 82.2733917236

In [61]:
tmp = np.empty([1, 28, 28, 2])
print(tmp.shape)

(1, 28, 28, 2)


In [122]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=False)
loss_object(y_true, y_pred).numpy()


8.17062

In [3]:
np.random.choice(5, 2, replace=False)


array([3, 0])